In [4]:
import altair as alt
import pandas as pd
import numpy as np

import datetime 

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [7]:
# This is the full, integrated COVID-19 data at UTLA level for England with cases, deaths, hospitalisations and vaccinations
# See the analyticalTeamDataProcessing notebook for the process that leads to this dataset
# note that the hospitalization data here is probabilistic at UTLA level

pheCovid = pd.read_csv('2021_08_04-pheCovid_England_UTLA_WithHospitalData.csv')
pheCovid.head()

,Unnamed: 0,areaCode,areaName,date,cumVaccinationCompleteCoverageByVaccinationDatePercentage,cumVaccinesGivenByPublishDate,newCasesByPublishDate,newDeaths28DaysByDeathDate,newHospitalisations
0,0,E06000003,Redcar and Cleveland,2021-08-04,NaN,NaN,62.0,NaN,0
1,1,E06000014,York,2021-08-04,NaN,NaN,135.0,NaN,0
2,2,E06000050,Cheshire West and Chester,2021-08-04,NaN,NaN,145.0,NaN,0
3,3,E08000001,Bolton,2021-08-04,NaN,NaN,76.0,NaN,0
4,4,E08000016,Barnsley,2021-08-04,NaN,NaN,130.0,NaN,0


In [22]:
chart = alt.Chart(pheCovid).mark_rect().encode(
    x='date:T',
    y='areaName:O',
    color=alt.Color('newHospitalisations:Q', scale=alt.Scale(scheme='yellowgreenblue')),
    tooltip=[
        alt.Tooltip('date:T', title='date'),
        alt.Tooltip('areaName', title='areaName'),
        alt.Tooltip('newHospitalisations:Q', title='newHospitalisations')
    ]).properties(width=1000, height=2000)

In [23]:
chart.save('chart.html')

## Relating to the rest of the PHE Indicator Data

Now that the full PHE data is available at UTLA level, we can relate this data to PHE's regular health indicator data set. We can use the Area Code fields to match these two data sets. However, the indicator data is huge so we'lll need to identify the indicators and also a time-window as well to filter this data.

In [6]:
phe_indicators_data = pd.read_csv('./Public Health Outcomes Framework/indicators-CountyUA.data.csv')
phe_indicators_data.head()

/Users/cagatayturkay/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Indicator ID,Indicator Name,Parent Code,Parent Name,Area Code,Area Name,Area Type,Sex,Age,Category Type,...,Upper CI 99.8 limit,Count,Denominator,Value note,Recent Trend,Compared to England value or percentiles,Compared to Region value or percentiles,Time period Sortable,New data,Compared to goal
0,90362,A01a - Healthy life expectancy at birth,NaN,NaN,E92000001,England,England,Male,All ages,NaN,...,NaN,NaN,NaN,NaN,NaN,Not compared,Not compared,20090000,NaN,NaN
1,90362,A01a - Healthy life expectancy at birth,NaN,NaN,E92000001,England,England,Female,All ages,NaN,...,NaN,NaN,NaN,NaN,NaN,Not compared,Not compared,20090000,NaN,NaN
2,90362,A01a - Healthy life expectancy at birth,E92000001,England,E12000001,North East region,Region,Male,All ages,NaN,...,NaN,NaN,NaN,NaN,NaN,Worse,Not compared,20090000,NaN,NaN
3,90362,A01a - Healthy life expectancy at birth,E92000001,England,E12000002,North West region,Region,Male,All ages,NaN,...,NaN,NaN,NaN,NaN,NaN,Worse,Not compared,20090000,NaN,NaN
4,90362,A01a - Healthy life expectancy at birth,E92000001,England,E12000003,Yorkshire and the Humber region,Region,Male,All ages,NaN,...,NaN,NaN,NaN,NaN,NaN,Worse,Not compared,20090000,NaN,NaN


In [16]:
# an example of relating to the indicators

# the meta-data has the information on these indicators

indicatorsChosen = ["Deprivation score (IMD 2019)"]
filteredIndicatorData = phe_indicators_data[phe_indicators_data['Indicator Name'].isin(indicatorsChosen)][["Area Code", "Value"]]

In [17]:
new_df = pd.merge(pheCovid, filteredIndicatorData,  how='left', left_on=['areaCode'], right_on = ['Area Code'])
# Drop the extra Area Code column from merging
new_df = new_df.drop(['Area Code'], axis=1)
#convert date to datetime
new_df.date = pd.to_datetime(new_df.date)

In [18]:
new_df.head()

,Unnamed: 0,areaCode,areaName,date,cumVaccinationCompleteCoverageByVaccinationDatePercentage,cumVaccinesGivenByPublishDate,newCasesByPublishDate,newDeaths28DaysByDeathDate,newHospitalisations,Area Code,Value
0,0,E06000003,Redcar and Cleveland,2021-08-04,NaN,NaN,62.0,NaN,0,E06000003,29.792
1,1,E06000014,York,2021-08-04,NaN,NaN,135.0,NaN,0,E06000014,11.727
2,2,E06000050,Cheshire West and Chester,2021-08-04,NaN,NaN,145.0,NaN,0,E06000050,18.083
3,3,E08000001,Bolton,2021-08-04,NaN,NaN,76.0,NaN,0,E08000001,30.691
4,4,E08000016,Barnsley,2021-08-04,NaN,NaN,130.0,NaN,0,E08000016,29.933
